#Tarea: Predicción Precio de Autos

# Nombre: Isaac Cueva

# Fecha: 14/05/2025

![Descripción de la imagen](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRP9d63jC144Usqa_NjecnoWcJXV60JuFTJLA&s)

#Google Colab Setup

In [1]:
# Installing required packages
!pip install pyspark
!pip install findspark

In [2]:
# PySpark is the Spark API for Python. In this lab, we use PySpark to initialize the spark context.
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
# Provides findspark.init() to make pyspark importable as a regular library.
import findspark
findspark.init()

In [3]:
# Creating a spark context class
sc = SparkContext()

# Creating a spark session
spark = SparkSession \
    .builder \
    .appName("Predicción de Autos") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [4]:
spark

#TAREAS

El conjunto de datos que se adjunta contiene información de marca, modelo, año de fabricación, precio, tipo de transmisión, kilometraje, tipo de combustible, impuesto de circulación, consumo en millas por galón (mpg) y tamaño del motor. Estos datos corresponden a ofertas de autos usados, listados en la web, en el Reino Unido.

Con los datos adjuntos, cree una jupyter notebook y complete las siguientes tarea:

1. Cree un dataframe de Spark que consolide toda la data.

In [8]:
from pyspark.sql.functions import lit

# Cargar los CSVs
bmw_df = spark.read.csv("bmw.csv", header=True, inferSchema=True)
ford_df = spark.read.csv("ford.csv", header=True, inferSchema=True)
hyundi_df = spark.read.csv("hyundi.csv", header=True, inferSchema=True)
merc_df = spark.read.csv("merc.csv", header=True, inferSchema=True)
skoda_df = spark.read.csv("skoda.csv", header=True, inferSchema=True)
toyota_df = spark.read.csv("toyota.csv", header=True, inferSchema=True)
vauxhall_df = spark.read.csv("vauxhall.csv", header=True, inferSchema=True)
vw_df = spark.read.csv("vw.csv", header=True, inferSchema=True)
audi_df = spark.read.csv("audi.csv", header=True, inferSchema=True)

# Función para renombrar la columna 'tax(£)' a 'tax' si existe
def rename_tax_column(df):
    if 'tax(£)' in df.columns:
        df = df.withColumnRenamed('tax(£)', 'tax')
    return df

# Aplicar renombramiento a cada DataFrame
bmw_df = rename_tax_column(bmw_df)
ford_df = rename_tax_column(ford_df)
hyundi_df = rename_tax_column(hyundi_df)
merc_df = rename_tax_column(merc_df)
skoda_df = rename_tax_column(skoda_df)
toyota_df = rename_tax_column(toyota_df)
vauxhall_df = rename_tax_column(vauxhall_df)
vw_df = rename_tax_column(vw_df)
audi_df = rename_tax_column(audi_df)

# Agregar columna "marca" con el nombre correspondiente
bmw_df = bmw_df.withColumn("marca", lit("bmw"))
ford_df = ford_df.withColumn("marca", lit("ford"))
hyundi_df = hyundi_df.withColumn("marca", lit("hyundi"))
merc_df = merc_df.withColumn("marca", lit("merc"))
skoda_df = skoda_df.withColumn("marca", lit("skoda"))
toyota_df = toyota_df.withColumn("marca", lit("toyota"))
vauxhall_df = vauxhall_df.withColumn("marca", lit("vauxhall"))
vw_df = vw_df.withColumn("marca", lit("vw"))
audi_df = audi_df.withColumn("marca", lit("audi"))

# Unir todos los DataFrames
from functools import reduce
from pyspark.sql import DataFrame

dfs = [bmw_df, ford_df, hyundi_df, merc_df, skoda_df, toyota_df, vauxhall_df, vw_df, audi_df]

def union_all(dfs):
    return reduce(DataFrame.unionByName, dfs)

autos_df = union_all(dfs)

# Mostrar esquema y algunas filas para verificar
autos_df.printSchema()
autos_df.show(5)


root
 |-- model: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- transmission: string (nullable = true)
 |-- mileage: integer (nullable = true)
 |-- fuelType: string (nullable = true)
 |-- tax: integer (nullable = true)
 |-- mpg: double (nullable = true)
 |-- engineSize: double (nullable = true)
 |-- marca: string (nullable = false)

+---------+----+-----+------------+-------+--------+---+----+----------+-----+
|    model|year|price|transmission|mileage|fuelType|tax| mpg|engineSize|marca|
+---------+----+-----+------------+-------+--------+---+----+----------+-----+
| 5 Series|2014|11200|   Automatic|  67068|  Diesel|125|57.6|       2.0|  bmw|
| 6 Series|2018|27000|   Automatic|  14827|  Petrol|145|42.8|       2.0|  bmw|
| 5 Series|2016|16000|   Automatic|  62794|  Diesel|160|51.4|       3.0|  bmw|
| 1 Series|2017|12750|   Automatic|  26676|  Diesel|145|72.4|       1.5|  bmw|
| 7 Series|2014|14500|   Automatic|  39554|  Diesel|16

2. Realice todo el pre-procesamiento de los datos que sean necesarios. No es necesario que se eliminen outliers.

- Valores Nulos

In [9]:
from pyspark.sql.functions import col, isnan, when, count
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline

# 1. Revisar cantidad de valores nulos por columna
autos_df.select([count(when(col(c).isNull() | isnan(col(c)), c)).alias(c) for c in autos_df.columns]).show()


+-----+----+-----+------------+-------+--------+---+---+----------+-----+
|model|year|price|transmission|mileage|fuelType|tax|mpg|engineSize|marca|
+-----+----+-----+------------+-------+--------+---+---+----------+-----+
|    0|   0|    0|           0|      0|       0|  0|  0|         0|    0|
+-----+----+-----+------------+-------+--------+---+---+----------+-----+



- Duplicados

In [11]:
#Contar valores duplicados (filas idénticas)
duplicados_count = autos_df.count() - autos_df.dropDuplicates().count()
print(f"Total de filas duplicadas: {duplicados_count}")

Total de filas duplicadas: 1475


In [12]:
total_filas = autos_df.count()
print(f"Total de filas en el DataFrame: {total_filas}")


Total de filas en el DataFrame: 99187


In [13]:
autos_df = autos_df.dropDuplicates()


- Definir Variables


In [14]:
#Definir columnas categóricas y numéricas
categorical_cols = ['transmission', 'fuelType', 'marca']
numeric_cols = ['year', 'mileage', 'tax', 'mpg', 'engineSize']

- Transformación de Variables

In [15]:
#Crear StringIndexers para variables categóricas
indexers = [StringIndexer(inputCol=col, outputCol=col + "_idx", handleInvalid='keep') for col in categorical_cols]

#OneHotEncoder para convertir índices en variables dummy
encoder = OneHotEncoder(inputCols=[col + "_idx" for col in categorical_cols],
                        outputCols=[col + "_oh" for col in categorical_cols])

#VectorAssembler para unir numéricas + variables dummy en 'features'
assembler = VectorAssembler(inputCols=numeric_cols + [col + "_oh" for col in categorical_cols],
                            outputCol="features")

- Ejecutar Pipeline en todo el dataset

In [16]:
#Crear Pipeline con los pasos: indexers -> encoder -> assembler
pipeline = Pipeline(stages=indexers + [encoder, assembler])

#Ajustar pipeline y transformar DataFrame
autos_prepared = pipeline.fit(autos_df).transform(autos_df)

#Mostrar resultado para verificar
autos_prepared.select("features", "price").show(5, truncate=False)

+--------------------------------------------------------------------+-----+
|features                                                            |price|
+--------------------------------------------------------------------+-----+
|(23,[0,1,2,3,4,5,9,18],[2019.0,3000.0,145.0,41.5,1.5,1.0,1.0,1.0])  |25898|
|(23,[0,1,2,3,4,6,10,18],[2018.0,5600.0,145.0,49.6,2.0,1.0,1.0,1.0]) |19990|
|(23,[0,1,3,4,6,10,18],[2016.0,31879.0,78.5,1.5,1.0,1.0,1.0])        |12998|
|(23,[0,1,2,3,4,7,10,18],[2016.0,39563.0,30.0,65.7,2.0,1.0,1.0,1.0]) |16380|
|(23,[0,1,2,3,4,6,10,18],[2018.0,20369.0,145.0,60.1,2.0,1.0,1.0,1.0])|21498|
+--------------------------------------------------------------------+-----+
only showing top 5 rows



3. Divida el dataframe en datos de entrenamiento y datos de evaluación (80/20).

In [17]:
# Renombrar 'price' a 'label' para que el modelo lo reconozca como variable objetivo
autos_prepared = autos_prepared.withColumnRenamed("price", "label")

# Dividir en 80% entrenamiento y 20% evaluación
train_df, test_df = autos_prepared.randomSplit([0.8, 0.2], seed=42)

# Verificar tamaños
print(f"Filas en entrenamiento: {train_df.count()}")
print(f"Filas en evaluación: {test_df.count()}")


Filas en entrenamiento: 78266
Filas en evaluación: 19446


4. Genere un modelo de regresión multilineal para predecir el precio de los autos.

In [18]:
from pyspark.ml.regression import LinearRegression

# Crear el modelo de regresión lineal
lr = LinearRegression(featuresCol="features", labelCol="label")

# Entrenar el modelo con el dataset de entrenamiento
lr_model = lr.fit(train_df)

# Mostrar coeficientes e intercepto
print("Coeficientes:", lr_model.coefficients)
print("Intercepto:", lr_model.intercept)


Coeficientes: [1496.534048417779,-0.0945050939518299,-3.2037993101880526,-56.36015305116176,9408.185400961025,-1246.2507694198057,1100.1318226282474,688.7447243753692,26.08142829761308,305.932104784459,-843.2997574936118,3877.947855434337,3622.385621018445,17261.886104379613,-429.0830897998381,352.01187413245407,-3447.219517443363,3057.498146560578,1043.9729261987761,3364.0164000531695,-2922.8296635660213,-1108.597104810715,-2212.469599453878]
Intercepto: -3011450.072881228


5. Calcule el valor de MSE y R^2 del modelo.

In [19]:
from pyspark.ml.evaluation import RegressionEvaluator

# Usar el modelo para predecir sobre el conjunto de prueba
predictions = lr_model.transform(test_df)

# Crear evaluador para MSE
evaluator_mse = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="mse")
mse = evaluator_mse.evaluate(predictions)

# Crear evaluador para R2
evaluator_r2 = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="r2")
r2 = evaluator_r2.evaluate(predictions)

print(f"Error cuadrático medio (MSE): {mse}")
print(f"Coeficiente de determinación (R²): {r2}")


Error cuadrático medio (MSE): 22696920.328210548
Coeficiente de determinación (R²): 0.7664911739703498


6. Genere un modelo de regresión polinomial de grado 2 (multivariable) para predecir el precio de los autos.

In [20]:
from pyspark.ml.feature import PolynomialExpansion
from pyspark.ml.regression import LinearRegression

# Crear expansión polinomial de grado 2
polyExpansion = PolynomialExpansion(degree=2, inputCol="features", outputCol="polyFeatures")

# Transformar el dataset de entrenamiento para agregar las características polinomiales
train_poly = polyExpansion.transform(train_df)
test_poly = polyExpansion.transform(test_df)

# Crear y entrenar el modelo de regresión lineal con las características polinomiales
lr_poly = LinearRegression(featuresCol="polyFeatures", labelCol="label")
lr_poly_model = lr_poly.fit(train_poly)

# Mostrar coeficientes e intercepto del modelo polinomial
print("Coeficientes polinomiales grado 2:", lr_poly_model.coefficients)
print("Intercepto polinomial grado 2:", lr_poly_model.intercept)


Coeficientes polinomiales grado 2: [572.1600579968283,0.16980844849522708,0.015610825868827748,5.201875806136374e-06,7.139476395064945e-07,6.150687788311893,0.0029868855412994192,2.22451379548923e-06,0.0541310506028858,-3.909196927287365,-0.0015216260679543488,-1.919281148642298e-05,-0.06394444598815828,-0.2043503780497102,3546.240351247642,1.8893900388900657,-0.10969536954510313,-19.937003828015058,-110.00474510550451,1718.6443845511913,-1877.1648456441599,-1.0218912623029373,0.018126076059687236,6.978995839500162,94.6205445797868,1926.9515787304958,-1877.1648456604748,1166.6427337843236,0.6535203710074999,-0.006486097577033363,-0.9958499630030652,-55.09654209758411,536.4793810641281,0.0,1166.6427338100953,1568.9232743109017,0.8371605913061434,0.000595203185251161,-1.879594075016725,-81.81994418484076,-591.3571084752256,0.0,0.0,1568.9232743303726,-74.81544373240523,-0.03691883838965228,-0.008287232144342583,8.821270966452033,-7.698525300572875,-1000.3321243809733,0.0,0.0,0.0,-74.81544

7. Calcule el valor de MSE y R^2 del modelo.

In [21]:
from pyspark.ml.evaluation import RegressionEvaluator

# Predecir con el modelo polinomial sobre conjunto de prueba
predictions_poly = lr_poly_model.transform(test_poly)

# Evaluador para MSE
evaluator_mse = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="mse")
mse_poly = evaluator_mse.evaluate(predictions_poly)

# Evaluador para R2
evaluator_r2 = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="r2")
r2_poly = evaluator_r2.evaluate(predictions_poly)

print(f"MSE modelo polinomial grado 2: {mse_poly}")
print(f"R² modelo polinomial grado 2: {r2_poly}")


MSE modelo polinomial grado 2: 13211241.709513092
R² modelo polinomial grado 2: 0.8640810516417066


8. Genere un modelo de regresión polinomial de grado 3 (multivariable) para predecir el precio de los autos.

In [22]:
from pyspark.ml.feature import PolynomialExpansion
from pyspark.ml.regression import LinearRegression

# Crear expansión polinomial de grado 3
polyExpansion3 = PolynomialExpansion(degree=3, inputCol="features", outputCol="polyFeatures3")

# Transformar datasets de entrenamiento y prueba
train_poly3 = polyExpansion3.transform(train_df)
test_poly3 = polyExpansion3.transform(test_df)

# Crear y entrenar el modelo de regresión lineal con características polinomiales grado 3
lr_poly3 = LinearRegression(featuresCol="polyFeatures3", labelCol="label")
lr_poly3_model = lr_poly3.fit(train_poly3)

# Mostrar coeficientes e intercepto
print("Coeficientes polinomiales grado 3:", lr_poly3_model.coefficients)
print("Intercepto polinomial grado 3:", lr_poly3_model.intercept)


Coeficientes polinomiales grado 3: [366.9193759039815,0.09824234557850804,3.486230373677955e-05,-0.010770922364930673,-5.792335538805363e-06,-3.0947617420712227e-09,8.594053175458204e-08,3.9231156466959545e-11,-2.4207651609620738e-12,4.123744683111939,0.002090658610027518,1.061029444911994e-06,-2.174330059005112e-05,-1.2082149730765587e-08,6.1601851559723e-10,-0.006548409479962662,-3.2682414648013948e-06,1.0681255244443269e-07,5.2319555023752515e-05,-25.395150190728895,-0.012610256395323637,-6.256379360039225e-06,0.00024858565887915713,1.1825624736552379e-07,-9.901541793814887e-10,0.022774943423097126,1.242775713963525e-05,9.839407241157205e-07,-0.0003617054393560804,0.06181440962510678,3.0749429140856944e-05,-1.3204443692223752e-07,-0.00042830913524435693,-0.0002563372456530778,2371.7239660572423,1.2111313650287128,0.0006180311240087931,-0.011236623572181604,-5.699132770221844e-06,4.1199283983345607e-07,0.21841069675838534,0.00017405632369450294,-1.394291726122965e-06,-0.0076433119917

9. Calcule el valor de MSE y R^2 del modelo.

In [23]:
from pyspark.ml.evaluation import RegressionEvaluator

# Predecir con el modelo polinomial grado 3 sobre el conjunto de prueba
predictions_poly3 = lr_poly3_model.transform(test_poly3)

# Crear evaluadores para MSE y R2
evaluator_mse = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="mse")
evaluator_r2 = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="r2")

# Calcular métricas
mse_poly3 = evaluator_mse.evaluate(predictions_poly3)
r2_poly3 = evaluator_r2.evaluate(predictions_poly3)

print(f"MSE modelo polinomial grado 3: {mse_poly3}")
print(f"R² modelo polinomial grado 3: {r2_poly3}")


MSE modelo polinomial grado 3: 11075720.635062357
R² modelo polinomial grado 3: 0.8860515662245492


10. Concluya cuál de los tres modelos tiene un mejor desempeño. Explique el por qué.

| Modelo                       | MSE           | R²     |
| ---------------------------- | ------------- | ------ |
| Regresión Lineal (grado 1)   | 22,696,920.33 | 0.7665 |
| Regresión Polinomial grado 2 | 13,211,241.71 | 0.8641 |
| Regresión Polinomial grado 3 | 11,075,720.64 | 0.8861 |

<br>

- Tras analizar los tres modelos generados para predecir el precio de autos usados, se concluye que el modelo de regresión polinomial de grado 3 presenta el mejor desempeño. Este modelo obtuvo el menor valor de Error Cuadrático Medio (MSE = 11,075,720.64) y el mayor coeficiente de determinación (R² = 0.8861), indicando una mejor capacidad para explicar la variabilidad del precio y un ajuste más preciso a los datos.

- Esto se debe a que el modelo de grado 3 permitió capturar relaciones no lineales y complejas entre las variables predictoras y el precio, mejorando sustancialmente la precisión del modelo.



11. ¿El mejor modelo obtenido es un buen modelo para usarlo en la práctica? Explique el por qué.

El modelo polinomial de grado 3 presenta un buen desempeño estadístico con un R² alto (0.886), lo que indica que explica bien la variabilidad del precio de los autos usados. Sin embargo, para usarlo en la práctica, es importante considerar que modelos tan complejos pueden sobreajustar los datos y ser menos interpretables. Por lo que es necesario validar su rendimiento con datos nuevos o mediante técnicas de validación cruzada.



12. Si quisieramos mejorar el rendimiento del mejor modelo, qué acciones tomaría.

Para mejorar el rendimiento del modelo se podrían considerar varias acciones:

- Aumentar la cantidad y calidad de datos: Más datos diversos y limpios ayudarán a que el modelo aprenda patrones más robustos y generales.
<br><br>
- Realizar ingeniería de características: Crear nuevas variables relevantes o transformar las existentes para capturar mejor las relaciones no lineales o interacciones.
<br><br>
- Regularización: Aplicar técnicas como Ridge o Lasso para evitar el sobreajuste y mejorar la generalización.
<br><br>
- Validación cruzada: Utilizar métodos como k-fold para ajustar hiperparámetros y evaluar el rendimiento de forma más fiable.
<br><br>
- Explorar otros modelos: Probar modelos más complejos o distintos (árboles, ensambles, redes neuronales) que puedan capturar relaciones más complejas.
<br><br>
- Optimización de hiperparámetros: Ajustar parámetros del modelo para mejorar su capacidad predictiva.

#APLICACIÓN DE VALIDACIÓN CRUZADA PARA MEJORAR EL MODELO

In [24]:
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import PolynomialExpansion
from pyspark.ml.regression import LinearRegression

# 1. Crear etapas pipeline
polyExpansion3 = PolynomialExpansion(degree=3, inputCol="features", outputCol="polyFeatures3")
lr = LinearRegression(featuresCol="polyFeatures3", labelCol="label")

pipeline = Pipeline(stages=[polyExpansion3, lr])

# 2. Crear grid de hiperparámetros para regParam y elasticNetParam
paramGrid = ParamGridBuilder() \
    .addGrid(lr.regParam, [0.0, 0.01, 0.1]) \
    .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0]) \
    .build()

# 3. Definir evaluador
evaluator = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")

# 4. Crear CrossValidator
crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=5)  # 5-fold CV

# 5. Ajustar CrossValidator con datos de entrenamiento
cvModel = crossval.fit(train_df)

# 6. Evaluar mejor modelo en test
predictions = cvModel.transform(test_df)
rmse = evaluator.evaluate(predictions)
r2 = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="r2").evaluate(predictions)

print(f"Mejor RMSE en validación cruzada: {rmse}")
print(f"R² en test con mejor modelo: {r2}")


Mejor RMSE en validación cruzada: 3328.020528041009
R² en test con mejor modelo: 0.8860515662245492


El modelo actual es muy bueno, y la regularización con validación cruzada ayudó a reducir el error de predicción y evitar el sobreajuste, pero para mejorar aún más conviene explorar nuevas variables, probar otros algoritmos y ampliar el dataset.